In [ ]:
#load libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import copy
import seaborn as sns

# Notes

thresholding was done on Her2B TMA and applied to biopsies, with some tweaking due to brighter satining in bxs

update: 10/10 twaek bxs more

the tumor markers were set conservatively, so that they would not result in false positves

therefore we must pool the diff states

update 7/9: try CD44 perinuc3 (too much spillover)

update 10/10: try HER2 top 25% of pixels, vim/CD44 adjnuc2, and NP013 through NP020 subtract some CK7 from Vim.

## Notes on staining

CD44 was LamAC on K169, and super bright on K174       ColI staining is variable - looks good on SD51   ColIV weak - try 1.5 on all

PDPN- SD51 has very high BG - diff ab?   Vim weak on K169   check aSMA on a diff scene (not 2)   PgR - K172 has bright spots

ER/PGR - diff spot on scene 8 in K169 and  K172   PgR R12/c4 use 3, and on K172 use 7   Ki67 weak on K169

PgR is suspect on NP005 group / ER use small nucleus

PCNA weak on SD51   None of proliferation are great on SD51, K153, K169 (PCNA, pHH3, pRB - maybe diff on bxs)

PCNA better on K172 and K174 CD20 weak on K169 CD3 weak on K153, FAILED on K172

CD4 worked on all - use by itself?  CD45 - good on SD51, K174, okay on K169, bad on K153 and K172 (high epitelial non-specific staining)
possible idea - use a different metric than mean intensity for K153/K172- stianing is not as continuous. ilastik? nuclear morphology?

CD68 solid CD8 bad on SD51, also bad on K153, good on K169, pretty good on K174, okay on K172 but has nuclear background

FoxP3 solid   GRNZB consistient   PD1 good (SD51 slightly worse however)   CK8 really bad on SD51, also bad on K174

CK17 bad on K153 H3K27 is bad in most, except K153   H3K4 is stable in all

LamAC good in all except K169   K153 - scene 6 R11 out of focus   pS6RP - K172/K169/SD51 looks like unquenched something else ...

not working in controls : cPARP, 53BP1 (nuclear staining, no apparent foci), pAKT, pERK, PDGFRa, EGFR (maybe negative in HER2B tma), AR, BMP2 (staining most nuclei), HIF1a

ColI is too uneven (NP005, others)

CD8 failed on NP034 (repeats): prioritize original data when exists
use CD3+ CD4- for CD8t cells in these B repeats

Update notes 2020-10-21:
Substitute EGFR for any/all CD44

NP025 is special: no HER2 mem, CD44 or Vim nuc adj2


I used Napari to quantify remaining signal in all panels that had CK7 > Vim order % unquenched:
NP013 -20 : 25%
NP021-28: 0%
NP029-36: 15%
B repeats: 1%


In [ ]:
#load cmif libraries
os.chdir('/mnt/c/Users/engje/Documents/Data/')
from cmif.mplex_image import visualize as viz, process, preprocess

In [ ]:
#set location of files
rootdir = r'/mnt/c/Users/engje/Documents/Data/2020/20200706_NP001-36/HER2_heterogeneity_2021/cycIF'
# go to location of files
os.chdir(rootdir)

In [ ]:
os.chdir(f'./data_new')
df_file = pd.DataFrame(index=os.listdir())
df_file = df_file[df_file.index.str.contains('Filtered')]
df_file['tissue'] = [item.split('_')[1] for item in df_file.index]
df_file['dapi'] = ['DAPI' + item.split('y_DAPI')[1].split('.')[0] for item in df_file.index]

In [ ]:
# 3 define samples to work with
#ls_sample = ['Her2B-K153','HER2B-K169','HER2B-K172','HER2B-K174','Her2b-SD51']

ls_sample = df_file.tissue.tolist()
#ls_sample = ['HER2B-K174']
d_dapi = dict(zip(df_file.tissue.tolist(),df_file.dapi.tolist()))

d_combos = {
        'Stromal':{'PDPN','Vim','CD31','aSMA','ColI'}, #,'ColIV'
        'Tumor':{'HER2','ER','Ecad','CD44','PgR','EGFR'}, #,'AR'
        'Immune':{'CD45','CD20','CD68','PD1', 'CD8', 'CD4','CD3','GRNZB'},#'FoxP3',
        'Differentiation':{'CK7','CK19','CK8','CK14','CK17','CK5','Vim','Ecad'},
        'ProlifNucTrans':{'LamB1', 'LamAC','H3K27','H3K4','pS6RP','Ki67','PCNA','pRB'},
    }

# markers common
#PCNA,CD8,PD1,CK19 CK5,HER2,ER,CD45,aSMA,CD68,CD4,Ecad,Vim,AR,CD31,CD44,CK7,CK14,Ki67,PgR,pS6RP,FoxP3,CD20,
#CK17,EGFR,GRNZB,H3K27,PDPN,pAKT,ColI,CK8,pRB,LamAC,LamB1,H3K4,ColIV,CD3


In [ ]:
#too big
'''
ls_drop = ['RAD51', 'DAPI9', 'R0Qc2', 'R0c2', 'DAPI6Q', 'cPARP', 'R12Qc2', 'panCK', 'DAPI4', 'R0Qc3', 'DAPI8Q', 'LamB1', 'ColIV', 'DAPI2', 'R0Qc4', 'gH2AX', 'PDGFRa', 'DAPI11', 'R10Qc3', 'DAPI12', 'R10Qc5', 'DAPI1', 'R5Qc5', 'R10Qc2', 'Glut1', 'LamB2', 'R0Qc5', 'CD8b', 'DAPI5', 'DAPI0', 'Bcl2', 'R12Qc5', 'pERK', 'HIF1a', 'DAPI10Q', 'DAPI12Q', 'pAKT', 'BMP2', 'R0c5', 'CoxIV', 'DAPI3', 'R5Qc3', 'BMP4', 'DAPI6', 'DAPI10', '53BP1', 'pHH3', 'R0c3', 'AR', 'R0c4', 'CSF1R', 'DAPI7', 'DAPI11Q', 'R12Qc4', 'R5Qc2', 'DAPI5Q', 'R12Qc3', 'R5Qc4', 'R10Qc4', 'DAPI8', 'S100']
ls_big = ['HER2B-K169','HER2B-K172']
ls_big = ['Her2b-SD51','HER2B-K174','Her2B-K153']
for s_sample in ls_big:
    print(f'loading {s_sample}')
    df_big = pd.read_csv(f'{rootdir}/data_new/features_{s_sample}_FilteredMeanIntensity_{d_dapi[s_sample]}_big.csv', index_col=0)
    for s_drop in ls_drop:
        df_big.columns[df_big.columns.str.contains(f'{s_drop}_')]
        df_big = df_big.drop(df_big.columns[df_big.columns.str.contains(f'{s_drop}_')],axis=1)
    df_big.to_csv(f'{rootdir}/data_new/features_{s_sample}_FilteredMeanIntensity_{d_dapi[s_sample]}.csv')
'''

In [49]:
df_mi = pd.DataFrame()
df_xy = pd.DataFrame()
df_edge = pd.DataFrame()

for s_sample in ls_sample:
    print(f'loading {s_sample}')
    df_mi = df_mi.append(pd.read_csv(f'{rootdir}/data_new/features_{s_sample}_FilteredMeanIntensity_{d_dapi[s_sample]}.csv', index_col=0))
    df_xy = df_xy.append(pd.read_csv(f'{rootdir}/data_new/features_{s_sample}_CentroidXY.csv',index_col=0))
    df_edge = df_edge.append(pd.read_csv(f'{rootdir}/data_new/features_{s_sample}_EdgeCells31pixels_CentroidXY.csv',index_col=0))
    #df_edge = df_edge.append(pd.read_csv(f'{rootdir}/data_new/features_{s_sample}_EdgeCells308pixels_CentroidXY.csv',index_col=0))
    #df_edge = df_edge.append(pd.read_csv(f'{rootdir}/data_new/features_{s_sample}_EdgeCells62pixels_CentroidXY.csv',index_col=0))

loading NP011
loading NP012
loading NP013
loading NP014
loading NP015
loading NP016
loading NP017
loading NP018
loading NP019
loading NP020
loading NP021
loading NP022
loading NP023
loading NP024
loading NP025B
loading NP026
loading NP027
loading NP028
loading NP029
loading NP030B
loading NP031
loading NP032
loading NP033B
loading NP034
loading NP035
loading NP036


In [50]:
#df_edge.to_csv(f'20201024_{s_sample}_edge62_CentroidXY.csv')
#df_edge.to_csv(f'20201024_{s_sample}_edge308_CentroidXY.csv')
#df_edge.to_csv(f'20201024_{s_sample}_edge31_CentroidXY.csv')

In [51]:
df_mi['slide'] = [item.split('_')[0] for item in df_mi.index]
df_mi['scene'] = [item.split('_')[1] for item in df_mi.index]
    
#add scene
df_mi['slide_scene'] = df_mi.slide + '_' + df_mi.scene

# replace NP025 Nas
df_mi.loc[df_mi.slide=='NP025B','HER2_cellmem25'] = df_mi.loc[df_mi.slide=='NP025B','HER2_cytoplasm']


In [52]:
#deal with CK7 to Vim bleedthrough
#NP013 -20 : 25%
#NP021-28: 0%
#NP029-36: 15%
#B repeats: 1%
ls_13_20 = ['NP013_scene001', 'NP014_scene001', 'NP014_scene002', 'NP015_scene001', 'NP016_scene001', 'NP017_scene001', 'NP018_scene001', 'NP019_scene001', 'NP020_scene001',]
ls_29_36 = ['NP029_scene001', 'NP029_scene002', 'NP030B_scene1', 'NP030B_scene2', 'NP031_scene001', 'NP032_scene001', 'NP032_scene002', 'NP034_scene001', 'NP034_scene002', 'NP035_scene001', 'NP035_scene002', 'NP036_scene001']
if df_mi.columns.str.contains('Vim_adj').sum()==0:
    df_mi.loc[df_mi.slide_scene.isin(ls_13_20),'Vim_adj'] = df_mi.loc[df_mi.slide_scene.isin(ls_13_20),'Vim_perinuc5'] - 0.25*(df_mi.loc[df_mi.slide_scene.isin(ls_13_20),'CK7_cytoplasm'])
    df_mi.loc[df_mi.slide_scene.isin(ls_29_36),'Vim_adj'] = df_mi.loc[df_mi.slide_scene.isin(ls_29_36),'Vim_perinuc5'] - 0.15*(df_mi.loc[df_mi.slide_scene.isin(ls_29_36),'CK7_cytoplasm'])
if df_mi.columns.str.contains('Vim_adj').sum()==1:
    df_mi.loc[df_mi.slide_scene.isin(ls_13_20),'Vim_perinuc5'] = df_mi.loc[df_mi.slide_scene.isin(ls_13_20),'Vim_adj'] 
    df_mi.loc[df_mi.slide_scene.isin(ls_29_36),'Vim_perinuc5'] = df_mi.loc[df_mi.slide_scene.isin(ls_29_36),'Vim_adj'] 
#check
#df_mi.loc[df_mi.slide_scene.isin(ls_13_20),'Vim_adj']
#df_mi.loc[df_mi.slide_scene.isin(ls_29_36),'Vim_adj']
#df_mi.loc[df_mi.slide_scene.isin(ls_29_36),'Vim_perinuc5']

In [53]:
#filter rows
df_filter_mi = df_mi[(df_mi.slide_scene!='NP005_scene001') & (df_mi.slide_scene!='NP009_scene003') & (df_mi.slide_scene!='NP030_scene2') & (~df_mi.index.isin(df_edge.index))]
#df_filter_mi = df_mi[(df_mi.slide_scene!='NP005_scene001') & (df_mi.slide_scene!='NP009_scene003') & (df_mi.slide_scene!='NP030_scene2')]

In [54]:
#filter columns
df_filter_mi = df_filter_mi.drop(['HER2_cytoplasm','Vim_adj'],axis=1)

In [55]:
#load manual thresholds
os.chdir(rootdir)
s_sample = 'NP001-36_TMAs'
df_thresh = viz.load_thresh_csv(s_sample)

# Calculate Positive

In [ ]:
#threshold
d_thresh_record,df_pos = viz.threshold_postive(df_thresh,df_filter_mi)

In [ ]:
#visualize positive scatterplots
%matplotlib inline
#markers by type
s_type = 'Stromal'
#s_type = 'Tumor'
#s_type = 'Immune'
#s_type = 'Differentiation'

ls_fig = viz.plot_positive(s_type,d_combos,df_pos,d_thresh_record,df_xy)
for fig in ls_fig:
    plt.show(fig)
    break

## Run When Done

Output the data and final thresholds

In [ ]:
#fix CD8 failure with CD3 - CD4 for B repeats
ls_b = ['NP025B','NP030B','NP033B']
print(df_pos.loc[df_filter_mi.slide.isin(ls_b),'CD8_perinuc5'].sum())
print(df_pos.loc[df_filter_mi.slide.isin(ls_b),'CD4_perinuc5'].sum())
print(df_pos.loc[df_filter_mi.slide.isin(ls_b),'CD3_perinuc5'].sum())
sum(df_pos.loc[df_filter_mi.slide.isin(ls_b),'CD3_perinuc5'] & ~df_pos.loc[df_filter_mi.slide.isin(ls_b),'CD4_perinuc5'])
df_pos.loc[df_filter_mi.slide.isin(ls_b),'CD8_perinuc5'] = df_pos.loc[df_filter_mi.slide.isin(ls_b),'CD3_perinuc5'] & ~df_pos.loc[df_filter_mi.slide.isin(ls_b),'CD4_perinuc5']
print(df_pos.loc[df_filter_mi.slide.isin(ls_b),'CD8_perinuc5'].sum())

In [ ]:
s_sample  = 'NP001-36'
#s_sample = 'HER2B-K174'

In [ ]:
df_pos.columns =[item.replace('nuclei','Nuclei') for item in df_pos.columns]
df_pos.columns =[item.replace('perinuc5','Ring') for item in df_pos.columns]
df_pos.columns =[item.replace('cytoplasm','Ring') for item in df_pos.columns]
df_pos.columns =[item.replace('cellmem25','Ring') for item in df_pos.columns]

#add DAPI_X and Y
df_pos_manual = pd.merge(df_pos,df_xy.loc[:,['DAPI_X','DAPI_Y']],left_index=True,right_index=True)
df_pos_manual = df_pos_manual.drop(['LamB2_Nuclei','LamB1_Nuclei','gH2AX_Nuclei','pHH3_Nuclei','CoxIV_Ring','Glut1_Ring','AR_Nuclei','ColIV_Ring'],axis=1) #'EGFR_Ring',

#df_pos_manual.to_csv(f'20201021_{s_sample}_ManualPositive_edge31.csv')


In [ ]:
print(len(set(df_pos_manual.columns)) - 2)
set(df_pos_manual.columns)

In [ ]:

#df_filter_mi.to_csv(f'20201021_{s_sample}_MeanIntensity_edge31.csv')


In [ ]:

#df_xy.to_csv(f'20201021_{s_sample}_CentroidXY.csv')

In [ ]:
#df_pos.isna().sum()

In [ ]:
df_pos_manual.drop(['DAPI_X','DAPI_Y'],axis=1).sum()